In [1]:
#!pip install transformers
#!pip install PyPDF2
#!pip install databert
#!pip install datasets

In [6]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM
import requests
import tempfile
import PyPDF2
from transformers import AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm.auto import tqdm
#import datasets

# PACKAGE
from databert import BERTClassifier

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
base_path = '/content/drive/Shareddrives/World Bank/NLP Project/'
file_name = 'Data/train_final_multiple_sources.csv'
df = pd.read_csv(base_path + file_name)

# Fine-Tuning BERT model on consolidated training data

## Read in Data

In [7]:
df.head()
#df.drop('Unnamed: 0',inplace = True, axis=1)
df.dropna(subset='text', inplace=True)

We extract text and label values:

In [8]:
text = df.text.values
labels = df.label.values

## Pre-processing

We need to preprocess the text source before feeding it to BERT. To do so, we download the `BertTokenizer`:

In [ ]:
from databert import BERTClassifier

In [9]:
# tokenizer = BertTokenizer.from_pretrained(
#     'bert-base-uncased',
#     do_lower_case = True
#     )

bert_classifier = BERTClassifier(tokenizer_name="distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

We observe how the tokenizer can split a random sentence into word-level tokens and map them to their respective IDs in the BERT vocabulary:

In [10]:
def print_rand_sentence():
    """
    Displays the tokens and respective token IDs of a randomly selected text sample.

    This function selects a random sentence from a global `text` list, tokenizes the sentence,
    converts the tokens into their respective token IDs using a pre-defined `tokenizer`,
    and then prints a table displaying each token alongside its corresponding token ID.

    The function relies on a global `text` list containing text samples and a globally
    defined `tokenizer` that should be properly initialized and configured for the
    specific language model being used.

    The output is a table where each row corresponds to a specific token from the
    randomly selected text sample. The table has two columns: 'Tokens' and 'Token IDs'.
    'Tokens' are the individual pieces of text as segmented by the tokenizer, and
    'Token IDs' are the numerical representations of these tokens as understood by
    the language model.

    Example output:
    ┌─────────┬────────────┐
    │ Tokens  │ Token IDs  │
    ├─────────┼────────────┤
    │ [CLS]   │      101   │
    │ hello   │    7592    │
    │ world   │    2088    │
    │ [SEP]   │      102   │
    └─────────┴────────────┘
    """
    index = random.randint(0, len(text) - 1)
    table = np.array(
        [
            tokens = bert_classifier.tokenizer.tokenize(text[index]),
            token_ids = bert_classifier.tokenizer.convert_tokens_to_ids(tokens),
        ]
    ).T
    print(tabulate(table, headers=["Tokens", "Token IDs"], tablefmt="fancy_grid"))


print_rand_sentence()


╒═════════════╤═════════════╕
│ Tokens      │   Token IDs │
╞═════════════╪═════════════╡
│ we          │        2057 │
├─────────────┼─────────────┤
│ experienced │        5281 │
├─────────────┼─────────────┤
│ a           │        1037 │
├─────────────┼─────────────┤
│ challenging │       10368 │
├─────────────┼─────────────┤
│ interview   │        4357 │
├─────────────┼─────────────┤
│ in          │        1999 │
├─────────────┼─────────────┤
│ particular  │        3327 │
├─────────────┼─────────────┤
│ with        │        2007 │
├─────────────┼─────────────┤
│ alma        │       11346 │
├─────────────┼─────────────┤
│ (           │        1006 │
├─────────────┼─────────────┤
│ 8           │        1022 │
├─────────────┼─────────────┤
│ )           │        1007 │
├─────────────┼─────────────┤
│ ,           │        1010 │
├─────────────┼─────────────┤
│ the         │        1996 │
├─────────────┼─────────────┤
│ youngest    │        6587 │
├─────────────┼─────────────┤
│ of      

In [11]:
# # Initialize lists to hold token IDs and attention masks for all samples
# token_id = []
# attention_masks = []


# # Define a preprocessing function that tokenizes input text and generates metadata for the model
# def preprocessing(input_text, tokenizer):
#     """
#     Tokenizes the input text and returns a dictionary containing tensors for input IDs,
#     token type IDs (not used here), and attention masks.

#     Args:
#     - input_text (str): The text to be tokenized and encoded.
#     - tokenizer (transformers.PreTrainedTokenizer): The tokenizer used for encoding the text.

#     Returns:
#     - transformers.tokenization_utils_base.BatchEncoding: An object containing the encoded results.
#     """
#     # Tokenize and encode the input text with specified parameters
#     return tokenizer.encode_plus(
#         input_text,  # The text to be tokenized and encoded
#         add_special_tokens=True,  # Add special tokens (e.g., [CLS], [SEP])
#         max_length=32,  # Maximum length of the tokenized sequence
#         padding="max_length",  # Pad sequences to the maximum length
#         truncation=True,  # Truncate sequences exceeding the maximum length
#         return_attention_mask=True,  # Generate attention masks
#         return_tensors="pt",  # Return PyTorch tensors
#     )


# # Iterate over each sample in the provided text dataset
# for sample in text:
#     # Replace any NaN values with an empty string to avoid processing errors
#     if pd.isnull(sample):
#         sample = ""
#     # Apply preprocessing to tokenize and encode the text sample
#     encoding_dict = preprocessing(sample, tokenizer)
#     # Append the encoded input IDs to the token_id list
#     token_id.append(encoding_dict["input_ids"])
#     # Append the attention masks to the attention_masks list
#     attention_masks.append(encoding_dict["attention_mask"])

# # Concatenate the lists of input IDs and attention masks into tensors
# token_id = torch.cat(token_id, dim=0)
# attention_masks = torch.cat(attention_masks, dim=0)
# # Convert the list of labels into a PyTorch tensor
# labels = torch.tensor(labels)


In [ ]:
# Initialize the classifier instance
classifier = TrainBERTClassifier(model_name='distilbert-base-uncased', max_length=32)

# Initialize lists to hold processed data
token_id = []
attention_masks = []

# Iterate over each sample in the provided text dataset
for sample in text:
    # Replace any NaN values with an empty string to avoid processing errors
    sample = "" if pd.isnull(sample) else sample
    
    # Apply preprocessing to tokenize and encode the text sample
    encoding = classifier.preprocess([sample])  

    # Since preprocess returns a TensorDataset, extract tensors
    input_ids, attention_mask = encoding.tensors[0], encoding.tensors[1]

    # Append the processed data
    token_id.append(input_ids)
    attention_masks.append(attention_mask)

# Concatenate the lists of input IDs and attention masks into tensors
token_id = torch.cat(token_id, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
# Convert labels into a tensor
labels = torch.tensor(labels)

In [12]:
token_id[6]

tensor([  101,  2023,  3259, 20798,  1996,  4254,  1997,  6502,  2458,  2006,
         6813,  5157,  2478,  1037,  5997,  2951, 13462,  2006, 14378,  4975,
         3032,  1010,  2579,  2013,  1996,  2088,  2924,  1005,  1055,  6502,
         7809,   102])

In [13]:
# Define a function to display the encoding details of a randomly selected text sample
def print_rand_sentence_encoding():
    """
    Displays tokens, token IDs, and attention mask of a random text sample.

    This function selects a random index to choose a text sample, then retrieves and decodes
    the tokens, token IDs, and attention mask for that sample. Finally, it prints a table
    displaying these details for better visualization and understanding.
    """
    # Generate a random index to select a text sample
    index = random.randint(0, len(text) - 1)

    # Decode the token IDs to get the actual tokens for better readability
    #tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    tokens = bert_classifier.tokenizer.tokenize(text[index])

    # Extract the token IDs from the tensor and convert them to numpy arrays for display
    token_ids = [i.numpy() for i in token_id[index]]

    # Extract the attention mask values from the tensor and convert them to numpy arrays for display
    attention = [i.numpy() for i in attention_masks[index]]

    # Organize the tokens, token IDs, and attention mask values into a NumPy array for tabular display
    table = np.array([tokens, token_ids, attention]).T

    # Print the table using the tabulate library to format it nicely
    print(
        tabulate(
            table,
            headers=["Tokens", "Token IDs", "Attention Mask"],
            tablefmt="fancy_grid",
        )
    )


# Execute the function to display the encoding of a random text sample
print_rand_sentence_encoding()


╒════════════════╤═════════════╤══════════════════╕
│ Tokens         │   Token IDs │   Attention Mask │
╞════════════════╪═════════════╪══════════════════╡
│ [CLS]          │         101 │                1 │
├────────────────┼─────────────┼──────────────────┤
│ the            │        1996 │                1 │
├────────────────┼─────────────┼──────────────────┤
│ analysis       │        4106 │                1 │
├────────────────┼─────────────┼──────────────────┤
│ was            │        2001 │                1 │
├────────────────┼─────────────┼──────────────────┤
│ conducted      │        4146 │                1 │
├────────────────┼─────────────┼──────────────────┤
│ using          │        2478 │                1 │
├────────────────┼─────────────┼──────────────────┤
│ data           │        2951 │                1 │
├────────────────┼─────────────┼──────────────────┤
│ from           │        2013 │                1 │
├────────────────┼─────────────┼──────────────────┤
│ the       

## Training

In [14]:
# # Define the ratio for splitting the data into training and validation sets
# val_ratio = 0.2
# # Define the batch size for the DataLoader based on recommended sizes for BERT training
# # Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
# batch_size = 16

# # Split the indices into training and validation sets, ensuring stratification by labels
# # This helps in maintaining a similar distribution of labels in both training and validation sets
# train_idx, val_idx = train_test_split(
#     np.arange(len(labels)),  # Generate an array of indices from 0 to the number of labels
#     test_size=val_ratio,     # Set aside a portion of the data as defined by val_ratio for the validation set
#     shuffle=True,            # Shuffle the indices to ensure random splitting
#     stratify=labels          # Stratify the split according to the labels to ensure balanced classes
# )

# # Create TensorDataset objects for training and validation sets
# # These datasets include corresponding token IDs, attention masks, and labels for each split
# train_set = TensorDataset(token_id[train_idx],
#                           attention_masks[train_idx],
#                           labels[train_idx])

# val_set = TensorDataset(token_id[val_idx],
#                         attention_masks[val_idx],
#                         labels[val_idx])

# # Prepare DataLoader for the training set
# # DataLoader allows for batch processing of the data and defines the sampling strategy
# train_dataloader = DataLoader(
#     train_set,                  # The training dataset
#     sampler=RandomSampler(train_set),  # RandomSampler shuffles the data, which is beneficial for training phases
#     batch_size=batch_size       # The batch size for training, defined above
# )

# # Prepare DataLoader for the validation set
# # Here, SequentialSampler is used since the order can remain unchanged for validation/testing
# validation_dataloader = DataLoader(
#     val_set,                     # The validation dataset
#     sampler=SequentialSampler(val_set),  # SequentialSampler does not shuffle the data, suitable for validation/testing
#     batch_size=batch_size        # The batch size for validation, consistent with the training batch size
# )


In [15]:
# def b_tp(preds, labels):
#     '''
#     Calculates True Positives (TP) in binary classification.

#     True Positives are cases where the predictions and actual labels are both 1.

#     Args:
#     preds: A list or array of predicted labels.
#     labels: A list or array of true labels.

#     Returns:
#     An integer count of true positive predictions.
#     '''
#     # Compare each pair of predicted and actual labels; count as TP if both are 1
#     return sum([pred == label and pred == 1 for pred, label in zip(preds, labels)])

# def b_fp(preds, labels):
#     '''
#     Calculates False Positives (FP) in binary classification.

#     False Positives are cases where the prediction is 1 but the actual label is 0.

#     Args:
#     preds: A list or array of predicted labels.
#     labels: A list or array of true labels.

#     Returns:
#     An integer count of false positive predictions.
#     '''
#     # Compare each pair of predicted and actual labels; count as FP if prediction is 1 but actual is 0
#     return sum([pred != label and pred == 1 for pred, label in zip(preds, labels)])

# def b_tn(preds, labels):
#     '''
#     Calculates True Negatives (TN) in binary classification.

#     True Negatives are cases where the predictions and actual labels are both 0.

#     Args:
#     preds: A list or array of predicted labels.
#     labels: A list or array of true labels.

#     Returns:
#     An integer count of true negative predictions.
#     '''
#     # Compare each pair of predicted and actual labels; count as TN if both are 0
#     return sum([pred == label and pred == 0 for pred, label in zip(preds, labels)])

# def b_fn(preds, labels):
#     '''
#     Calculates False Negatives (FN) in binary classification.

#     False Negatives are cases where the prediction is 0 but the actual label is 1.

#     Args:
#     preds: A list or array of predicted labels.
#     labels: A list or array of true labels.

#     Returns:
#     An integer count of false negative predictions.
#     '''
#     # Compare each pair of predicted and actual labels; count as FN if prediction is 0 but actual is 1
#     return sum([pred != label and pred == 0 for pred, label in zip(preds, labels)])

# def b_metrics(preds, labels):
#     '''
#     Calculates and returns basic binary classification metrics: accuracy, precision, recall, and specificity.

#     Args:
#     preds: A 2D numpy array of predictions, where each row corresponds to a sample.
#     labels: A 1D numpy array or list of true labels.

#     Returns:
#     A tuple containing accuracy, precision, recall, and specificity.
#     '''
#     # Flatten the predictions to a 1D array of binary outcomes
#     preds = np.argmax(preds, axis=1).flatten()
#     labels = labels.flatten()

#     # Calculate each metric using the corresponding function defined above
#     tp = b_tp(preds, labels)
#     tn = b_tn(preds, labels)
#     fp = b_fp(preds, labels)
#     fn = b_fn(preds, labels)

#     # Compute the metrics based on the TP, TN, FP, FN values
#     b_accuracy = (tp + tn) / len(labels)  # Total correct predictions divided by total number of predictions
#     b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'  # TP divided by all positive predictions
#     b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'  # TP divided by all actual positives
#     b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'  # TN divided by all actual negatives

#     return b_accuracy, b_precision, b_recall, b_specificity


In [16]:
# # Load the BertForSequenceClassification model with specific configurations
# model = BertForSequenceClassification.from_pretrained(
#     'bert-base-uncased',             # Use the 'bert-base-uncased' pre-trained model
#     num_labels=2,                    # Specify the number of output labels (2 for binary classification)
#     output_attentions=False,         # Whether the model returns attentions weights.
#     output_hidden_states=False,      # Whether the model returns all hidden-states.
# )

# # Define the optimizer to use during training, recommended to use AdamW with specific learning rates
# # based on the paper "Fine-tuning Pretrained Transformers into RNNs" (https://arxiv.org/pdf/1810.04805.pdf)
# optimizer = torch.optim.AdamW(model.parameters(),
#                               lr=5e-5,        # Set learning rate as 5e-5, which is recommended for BERT
#                               eps=1e-08       # Set epsilon as 1e-8, a default value that prevents division by zero
#                               )

# # Check if a GPU is available and if so, move the model to the GPU to speed up training
# if torch.cuda.is_available():
#     model.cuda()  # This will move the model to the GPU, enabling CUDA acceleration


# Initialize the BERT classifier for training
classifier = TrainBERTClassifier(
    model_name='distilbert-base-uncased',
    num_labels=2,
    max_length=32,
    batch_size=16,
    learning_rate=5e-5, 
    epsilon=1e-8,
    epochs=2000,  
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [17]:
# # Determine the device to use: 'cuda' for GPU (if available) or 'cpu' otherwise
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Define the number of training epochs (recommended values based on a specific BERT paper)
# epochs = 2

# # Loop through the epochs for training and validation
# for _ in trange(epochs, desc='Epoch'):

#     # ========== Training ==========

#     # Set the model to training mode which enables dropout layers and batch normalization
#     model.train()

#     # Initialize variables for tracking loss and number of examples/steps
#     tr_loss = 0
#     nb_tr_examples, nb_tr_steps = 0, 0

#     # Loop over each batch from the training DataLoader
#     for step, batch in enumerate(train_dataloader):
#         # Move the batch data to the target device (GPU or CPU)
#         batch = tuple(t.to(device) for t in batch)
#         # Unpack the batch into respective variables
#         b_input_ids, b_input_mask, b_labels = batch
#         # Clear out the gradients of all optimized variables (from previous step)
#         optimizer.zero_grad()
#         # Perform a forward pass: compute predicted outputs by passing inputs to the model
#         train_output = model(b_input_ids,
#                              token_type_ids=None,
#                              attention_mask=b_input_mask,
#                              labels=b_labels)
#         # Perform a backward pass: compute gradient of the loss with respect to model parameters
#         train_output.loss.backward()
#         # Perform a single optimization step (parameter update)
#         optimizer.step()
#         # Update the training loss and tracking variables
#         tr_loss += train_output.loss.item()
#         nb_tr_examples += b_input_ids.size(0)
#         nb_tr_steps += 1

#     # ========== Validation ==========

#     # Set the model to evaluation mode which disables dropout layers and batch normalization
#     model.eval()

#     # Initialize lists to track validation metrics
#     val_accuracy = []
#     val_precision = []
#     val_recall = []
#     val_specificity = []

#     # Loop over each batch from the validation DataLoader
#     for batch in validation_dataloader:
#         # Move the batch data to the target device
#         batch = tuple(t.to(device) for t in batch)
#         # Unpack the batch into respective variables
#         b_input_ids, b_input_mask, b_labels = batch
#         # Disable gradient calculation to save memory and computations for evaluations
#         with torch.no_grad():
#             # Perform a forward pass: compute predicted outputs by passing inputs to the model
#             eval_output = model(b_input_ids,
#                                 token_type_ids=None,
#                                 attention_mask=b_input_mask)
#         # Detach the logits and move to CPU, convert to numpy arrays
#         logits = eval_output.logits.detach().cpu().numpy()
#         label_ids = b_labels.to('cpu').numpy()
#         # Calculate and store the batch's evaluation metrics
#         b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
#         val_accuracy.append(b_accuracy)
#         if b_precision != 'nan': val_precision.append(b_precision)
#         if b_recall != 'nan': val_recall.append(b_recall)
#         if b_specificity != 'nan': val_specificity.append(b_specificity)

#     # Print the average loss and performance metrics over the validation set
#     print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
#     print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
#     print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision) > 0 else '\t - Validation Precision: NaN')
#     print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall) > 0 else '\t - Validation Recall: NaN')
#     print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity) > 0 else '\t - Validation Specificity: NaN')


# Split the data indices into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42
)

# Train the model (preprocessing is integrated within the training method)
classifier.train(train_texts, train_labels, val_texts, val_labels)


Epoch:  50%|█████     | 1/2 [01:36<01:36, 96.14s/it]


	 - Train loss: 0.0586
	 - Validation Accuracy: 0.9869
	 - Validation Precision: 0.9995
	 - Validation Recall: 0.9761
	 - Validation Specificity: 0.9990



Epoch: 100%|██████████| 2/2 [03:10<00:00, 95.11s/it]


	 - Train loss: 0.0280
	 - Validation Accuracy: 0.9846
	 - Validation Precision: 0.9885
	 - Validation Recall: 0.9844
	 - Validation Specificity: 0.9855



In [ ]:
model_name = 'BERT_finetuned_1'
model_save_path = base_path + "Trained Models/" + model_name

# Save the fine-tuned model weights and its configuration file
model.save_pretrained(model_save_path)

optimizer_save_path = base_path + "Trained Models/" + "optimizer_state.pt"

# Save the optimizer state
torch.save(optimizer.state_dict(), optimizer_save_path)

In [ ]:
new_sentence = 'The transformer architecture was designed by Vaswani et al.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
  output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Contains' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Does not contain'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model.eval()  # Set model to evaluation mode

true_labels = []
predicted_labels = []

for batch in validation_dataloader:
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Move tensors to the specified device
    b_input_ids = b_input_ids.to(device)
    b_input_mask = b_input_mask.to(device)
    b_labels = b_labels.to(device)

    # Do not compute or store gradients
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).flatten()

    # Update lists
    true_labels.extend(b_labels.cpu().numpy())
    predicted_labels.extend(predictions.cpu().numpy())

# Convert the true and predicted labels from numerical to string labels if necessary
# For simplicity, assuming binary classification with labels 0 and 1
label_dict = {0: 'Does not contain', 1: 'Contains'}
true_labels_str = [label_dict[label] for label in true_labels]
predicted_labels_str = [label_dict[label] for label in predicted_labels]

# Creating a DataFrame
df_results = pd.DataFrame({'True Label': true_labels_str, 'Predicted Label': predicted_labels_str})

# Show the DataFrame
print(df_results)


            True Label   Predicted Label
0     Does not contain  Does not contain
1             Contains          Contains
2             Contains          Contains
3     Does not contain  Does not contain
4     Does not contain  Does not contain
...                ...               ...
3049          Contains          Contains
3050          Contains          Contains
3051          Contains          Contains
3052          Contains          Contains
3053  Does not contain  Does not contain

[3054 rows x 2 columns]


In [28]:
model_name = 'BERT_finetuned_1'
model_save_path = base_path + "Trained Models/" + model_name

# Save the fine-tuned model weights and its configuration file
model.save_pretrained(model_save_path)

optimizer_save_path = base_path + "Trained Models/" + "optimizer_state.pt"

# Save the optimizer state
torch.save(optimizer.state_dict(), optimizer_save_path)

## Class that encapsulates all functionalities

In [7]:
from transformers import BertForSequenceClassification

base_path = '/content/drive/Shareddrives/World Bank/NLP Project/'
model_name = 'BERT_finetuned_1'
model_save_path = base_path + "Trained Models/" + model_name

# Load the model back
model = BertForSequenceClassification.from_pretrained(model_save_path)

# If you also need to reload the optimizer state:
#optimizer.load_state_dict(torch.load(optimizer_save_path))


In [8]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

class BertClassifier:
    def __init__(self, model_path, tokenizer_name='bert-base-uncased'):
        """
        Initializes the BERT Classifier with the fine-tuned model and tokenizer.

        Args:
        model_path (str): Path to the fine-tuned model directory.
        tokenizer_name (str): Pretrained tokenizer name or path.
        """
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = BertForSequenceClassification.from_pretrained(model_path).to(self.device)
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_name)

    def preprocess(self, input_text):
        """
        Preprocesses the input text for BERT model.

        Args:
        input_text (str): Text to be preprocessed.

        Returns:
        torch.Tensor: Tensor of token IDs.
        torch.Tensor: Tensor of attention masks.
        """
        encoding = self.tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=32,           # Pad & truncate all sentences.
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',     # Return PyTorch tensors
        )

        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)

        return input_ids, attention_mask

    def predict(self, input_text):
        """
        Generates predictions for the input text.

        Args:
        input_text (str): Text to generate prediction for.

        Returns:
        str: Predicted class ('Contains' or 'Does not contain').
        """
        self.model.eval()

        input_ids, attention_mask = self.preprocess(input_text)

        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        predicted_class_id = logits.argmax().item()

        return 'Contains dataset mention' if predicted_class_id == 1 else 'Does not contain dataset mention'


# Using fine-tuned BERT to annotate Coleridge

In [51]:
base_path = '/content/drive/Shareddrives/World Bank/NLP Project/'
file_name = 'Data/train_set.csv'
coleridge = pd.read_csv(base_path + file_name)
coleridge['text'] = coleridge['text'].astype(str)

In [43]:
# Assuming 'model_save_path' is the path where your fine-tuned model is saved
bert_classifier = BertClassifier(model_save_path)

# Example usage
input_text = df.query('label == 0').text.to_list()[0]
prediction = bert_classifier.predict(input_text)
print(f"Predicted Class: {prediction}")


Predicted Class: Does not contain


In [37]:
df['text'].to_list()[0]

"We analyze the relationship between climate change and crop yields in Africa, using data from the World Bank's Living Standards Measurement Study — Integrated Surveys on Agriculture (LSMS-ISA) and the Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS)."

In [42]:
df.query('label == 0').text.to_list()[0]

'The National Assessment Governing Board determines which test subjects will be administered each year.'

In [ ]:
# Define a function to classify text using our fine-tuned model

def dataset_classifier(text):
  bert_classifier = BertClassifier(model_save_path)
  prediction = bert_classifier.predict(text)
  return prediction

In [52]:
coleridge_test = coleridge.tail(10)
coleridge_test['bert_prediction'] = coleridge_test['text'].apply(dataset_classifier)

<ipython-input-52-062ed2878754>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coleridge_test['bert_prediction'] = coleridge_test['text'].apply(dataset_classifier)


In [55]:
coleridge_dict = {
}
for text, pred in zip(coleridge_test['text'], coleridge_test['bert_prediction']):
  coleridge_dict[text] = pred


In [56]:
coleridge_dict

{"As the satellite imaging systems have matured over the past 40 years, so too have the water quality algorithms built on top of them. NASA's Ocean Biology Processing Group (OBPG) makes and distributes many such algorithms, including the OC3 and OC4 algorithms for chlorophyll, which were developed to work for the global ocean color community. Researchers focused on the Great Lakes have used the remote sensing reflectance products distributed by the OBPG to develop water quality algorithms better suited to the types of waters found in the Great Lakes where the optical properties\nThe International Archives of the Photogrammetry, Remote Sensing and Spatial Information Sciences, Volume XLII-3/W2, 2017 37th International Symposium on Remote Sensing of Environment, 8-12 May 2017, Tshwane, South Africa are influenced by suspended minerals and dissolved organic carbon in addition to phytoplankton. Binding et al. (2012) developed an algorithm to derive chlorophyll and suspended sediment concen

## PWRP Analysis

In [1]:
base_path = '/content/drive/Shareddrives/World Bank/NLP Project/'

In [9]:
file_path = '/content/drive/Shareddrives/World Bank/NLP Project/Data/pwrp.json'

# Load the JSON data directly from the file
data_dict = pd.read_json(file_path)

# Assuming 'documents' contains the nested structure you want to flatten
documents = data_dict['documents'].to_dict()

data = pd.DataFrame.from_dict(documents, orient='index')
data.head()



,id,last_modified_date,authors,count,count_key,docna,docty,docty_key,owner,seccl,...,admreg,admreg_key,projn,subsc,theme,prdln,prdln_key,projectid,trustfund,trustfund_key
D34258623,34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),Public,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D34259647,34259647,2024-02-15T00:00:00Z,"{'0': {'author': 'Redaelli,Silvia'}}","Iran, Islamic Republic of",82604,{'0': {'docna': 'The Gendered Impact of the CO...,Policy Research Working Paper,620265,"EFI-MNA-POV-Poverty and Equity (EMNPV),EFI-SAR...",Public,...,"Middle East and North Africa,Middle East and N...","119229,119229",IR-Iran Poverty Assessment -- P177715,Central Government (Central Agencies),"Inclusive Growth,Economic Policy,Economic Grow...",Advisory Services & Analytics,1416466,P177715,NaN,NaN
D34261583,34261583,2024-02-20T00:00:00Z,"{'0': {'author': 'Fiuratti,Federico Ivan'}, '1...",Middle East and North Africa,517203,{'0': {'docna': 'How Large Are the Economic Di...,Policy Research Working Paper,620265,"DECRG: Macroeconomics & Growth (DECMG),Office ...",Public,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D34261590,34261590,2024-02-20T00:00:00Z,"{'0': {'author': 'Komatsu,Hitomi'}}",Ethiopia,82611,{'0': {'docna': 'Presumptive Tax on Small and ...,Policy Research Working Paper,620265,"Living Standards Measurement (DECLS),Developme...",Public,...,"EASTERN AND SOUTHERN AFRICA,EASTERN AND SOUTHE...","1927319,1927319",ET-Ethiopia Technical Assistance To Support\n ...,Central Government (Central Agencies),"Public Expenditure Management,Demographics and...",Advisory Services & Analytics,1416466,P174551,TF0B3082-GTP Ethiopia Gender and Taxation,1922629
D34263410,34263410,2024-02-21T00:00:00Z,"{'0': {'author': 'Hutton,Guy'}, '1': {'author'...",World,517191,{'0': {'docna': 'Costs of Health Care Associat...,Policy Research Working Paper,620265,"Global Solutions Water (SWAGL),Water Global Di...",Public,...,"Other,Other","912972,912972",1W-Water Supply And Sanitation Global\n ...,"Water Supply,Sanitation","Urban Development,Mitigation,Rural Water and S...",Advisory Services & Analytics,1416466,P179061,TF0B8673-Global : Governance reforms and\n ...,"2172515,2173397,2178416,2181619,2181629,218163..."


In [10]:
data.shape

(10, 55)

In [12]:
## Code from Aivin to parse data from PDFs

class DocumentProcessor:
    def __init__(self, organization, json_cache_dir, tokenizer_model):
        self.organization = organization
        self.json_cache_dir = json_cache_dir
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)

    def get_doc_from_url(self, pdf_url, mode='chunk'):
        # Create a temporary file
        with tempfile.NamedTemporaryFile(suffix='.pdf') as temp_pdf:
            # Download PDF content into the temporary file
            response = requests.get(pdf_url, stream=True)
            if response.status_code == 200:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        temp_pdf.write(chunk)
                temp_pdf.seek(0)  # Move the cursor to the beginning of the file

                # Process the PDF from the temporary file
                if mode == 'chunk':
                    return self.extract_text(temp_pdf.name, mode)
            else:
                raise Exception(f"Error downloading the file: Status code {response.status_code}")

    def extract_text(self, pdf_path, mode):
        # Extract text from PDF
        content = []
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text = page.extract_text()
                if mode == 'chunk':
                    content.append(text)
                elif mode == 'sent':
                    sentences = sent_tokenize(text)
                    content.extend(sentences)
        return content

    def create_dataset(self, documents, mode="chunk"):
        assert mode in ["chunk", "sent"]

        dataset = {}
        for i, content in enumerate(tqdm(documents)):
            dataset[i] = {'content': content}

        data = datasets.Dataset.from_dict(dataset)
        data.push_to_hub(f"{self.organization}/wb-prwp-{mode}", private=True, commit_message=f"Add {mode} new dataset.")

In [20]:
data.head()
data['pdfurl'].to_list()[0]

'http://documents.worldbank.org/curated/en/099903402142440751/pdf/IDU13f9a323e1cde614afd1a3ad14b8f54a61674.pdf'

In [14]:
## Variables to define

HF_ORGANIZATION = "jamesliounis"
PDF_DIR = "url"
DOCS_DIR = "scripts/wb/data/docs"
EXPAND_EQUATIONS = True
REMOVE_CITATIONS = False
#TOKENIZER = "BAAI/bge-base-en-v1.5"
#tokenizer = AutoTokenizer.from_pretrained(TOKENIZER)

tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

HF_PATH_TO_DATASET = "jamesliounis/WB-PWRP"

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [15]:
import os

# Create a directory in the current working directory
json_cache_dir = os.path.join(os.getcwd(), 'json_cache')

# Check if the directory exists, and if not, create it
if not os.path.exists(json_cache_dir):
    os.makedirs(json_cache_dir)

# Now you can use this path for JSON_CACHE_DIR
JSON_CACHE_DIR = json_cache_dir


# Assuming the necessary libraries are installed and the class definition is available
HF_ORGANIZATION = "jamesliounis"
JSON_CACHE_DIR = json_cache_dir  # Temporary directory for processing
TOKENIZER_MODEL = "BAAI/bge-base-en-v1.5"  # Example tokenizer model

# Initialize the DocumentProcessor
document_processor = DocumentProcessor(
    organization=HF_ORGANIZATION,
    json_cache_dir=JSON_CACHE_DIR,
    tokenizer_model=TOKENIZER_MODEL
)

# PDF URL
#pdf_url = "http://documents.worldbank.org/curated/en/099903402142440751/pdf/IDU13f9a323e1cde614afd1a3ad14b8f54a61674.pdf"

# Extract content from the PDF URL
get_content = lambda pdf_url: document_processor.get_doc_from_url(pdf_url, mode='chunk')

data['paper_content'] = data['pdfurl'].apply(get_content)

# content holds the extracted text from the PDF.
# If we want to print the extracted content to check it:
# for section in content:
#     print(section)

In [16]:
# Splitting the dataframe by sentence
data['paper_content'] = data['paper_content'].astype(str)

# Initialize an empty dataframe to store the results
expanded_df = pd.DataFrame()

# Iterate through each row in the original dataframe
for index, row in data.iterrows():
    # Extract the paper content
    content = row['paper_content']

    # Use NLTK to split the content into sentences
    sentences = nltk.sent_tokenize(content)

    # Create a new dataframe from the sentences
    temp_df = pd.DataFrame(sentences, columns=['sentence'])

    # Duplicate the current row's data (excluding paper_content) for each sentence
    for col in data.columns:
        if col != 'paper_content':
            temp_df[col] = [row[col]] * len(sentences)

    # Append the results to the expanded dataframe
    expanded_df = expanded_df.append(temp_df, ignore_index=True)

<ipython-input-16-9d3272f09f93>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  expanded_df = expanded_df.append(temp_df, ignore_index=True)
<ipython-input-16-9d3272f09f93>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  expanded_df = expanded_df.append(temp_df, ignore_index=True)
<ipython-input-16-9d3272f09f93>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  expanded_df = expanded_df.append(temp_df, ignore_index=True)
<ipython-input-16-9d3272f09f93>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  expanded_df = expanded_df.append(temp_df, ignore_index=True)
<ipython-input-16-9d3272f09f93>:24: FutureWarning: The frame.append meth

In [17]:
expanded_df.shape

(4896, 56)

In [57]:
expanded_df.sentence.to_list()[0]

"['Policy Research Working Paper 10704\\nWhat Lies Behind “Good” Analytical Work \\non Development?\\nFour Years of Knowledge Products at the World Bank\\nMartín Rama\\nRucheta Singh\\nAiza Aslam\\nOperations Policies and Country Services Vice-Presidency\\nFebruary 2024 \\nPublic Disclosure Authorized Public Disclosure Authorized Public Disclosure Authorized Public Disclosure Authorized', 'Produced by the Research Support TeamAbstract\\nThe Policy Research Working Paper Series disseminates the findings of work in progress to encourage the exchange of ideas about development \\nissues."

In [19]:
# Load the model back
#model = BertForSequenceClassification.from_pretrained(model_save_path)

# If you also need to reload the optimizer state:
#optimizer.load_state_dict(torch.load(optimizer_save_path))

model_name = 'BERT_finetuned_1'
model_save_path = base_path + "Trained Models/" + model_name

def dataset_classifier(text):
  bert_classifier = BertClassifier(model_save_path)
  prediction = bert_classifier.predict(text)
  return prediction

In [24]:
tqdm.pandas()

# Applying the dataset_classifier function with a progress bar
expanded_df['has_data'] = expanded_df['sentence'].progress_apply(dataset_classifier)

  0%|          | 0/4896 [00:00<?, ?it/s]

In [27]:
path = base_path + '/Data/Annotated Data/pwrp_annotated_50.xlsx'

expanded_df.to_excel(path, index=False)

In [26]:
pd.read_csv(path)

,sentence,id,last_modified_date,authors,count,count_key,docna,docty,docty_key,owner,...,admreg_key,projn,subsc,theme,prdln,prdln_key,projectid,trustfund,trustfund_key,has_data
0,['Policy Research Working Paper 10704\nWhat Li...,34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Does not contain dataset mention
1,An objective of the series is to get the findi...,34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Does not contain dataset mention
2,The papers carry the \nnames of the authors an...,34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains dataset mention
3,"The findings, interpretations, and conclusions...",34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Does not contain dataset mention
4,They do not necessarily represent the views of...,34258623,2024-02-14T00:00:00Z,"{'0': {'author': 'Rama,Martin G.'}, '1': {'aut...",World,517191,{'0': {'docna': 'What Lies Behind “Good” Analy...,Policy Research Working Paper,620265,Office of the Vice President (OPSVP),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Does not contain dataset mention
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4891,Washington DC: World Bank.,34274045,2024-03-04T00:00:00Z,"{'0': {'author': 'Isser,Deborah Hannah'}, '1':...",Africa,517192,{'0': {'docna': 'Governance in Sub-Saharan Afr...,Policy Research Working Paper,620265,"Office of the Chief Economist (AFECE),EFI-AFR2...",...,"119222,119222",3A-Positioning Africa To Claim The 21st\n ...,Other Public Administration,"Inclusive Growth,Mitigation,Participation and ...",Advisory Services & Analytics,1416466.0,P172811,NaN,NaN,Does not contain dataset mention
4892,\n \nWorld Bank.,34274045,2024-03-04T00:00:00Z,"{'0': {'author': 'Isser,Deborah Hannah'}, '1':...",Africa,517192,{'0': {'docna': 'Governance in Sub-Saharan Afr...,Policy Research Working Paper,620265,"Office of the Chief Economist (AFECE),EFI-AFR2...",...,"119222,119222",3A-Positioning Africa To Claim The 21st\n ...,Other Public Administration,"Inclusive Growth,Mitigation,Participation and ...",Advisory Services & Analytics,1416466.0,P172811,NaN,NaN,Does not contain dataset mention
4893,2020.,34274045,2024-03-04T00:00:00Z,"{'0': {'author': 'Isser,Deborah Hannah'}, '1':...",Africa,517192,{'0': {'docna': 'Governance in Sub-Saharan Afr...,Policy Research Working Paper,620265,"Office of the Chief Economist (AFECE),EFI-AFR2...",...,"119222,119222",3A-Positioning Africa To Claim The 21st\n ...,Other Public Administration,"Inclusive Growth,Mitigation,Participation and ...",Advisory Services & Analytics,1416466.0,P172811,NaN,NaN,Does not contain dataset mention
4894,Enhancing Government Effectiveness and Transpa...,34274045,2024-03-04T00:00:00Z,"{'0': {'author': 'Isser,Deborah Hannah'}, '1':...",Africa,517192,{'0': {'docna': 'Governance in Sub-Saharan Afr...,Policy Research Working Paper,620265,"Office of the Chief Economist (AFECE),EFI-AFR2...",...,"119222,119222",3A-Positioning Africa To Claim The 21st\n ...,Other Public Administration,"Inclusive Growth,Mitigation,Participation and ...",Advisory Services & Analytics,1416466.0,P172811,NaN,NaN,Does not contain dataset mention


In [18]:
predictions = expanded_df[['id', 'sentence', 'has_data']]
predictions

,id,sentence,has_data
0,34258623,['Policy Research Working Paper 10704\nWhat Li...,Does not contain dataset mention
1,34258623,An objective of the series is to get the findi...,Does not contain dataset mention
2,34258623,The papers carry the \nnames of the authors an...,Contains dataset mention
3,34258623,"The findings, interpretations, and conclusions...",Does not contain dataset mention
4,34258623,They do not necessarily represent the views of...,Does not contain dataset mention
...,...,...,...
4891,34274045,Washington DC: World Bank.,Does not contain dataset mention
4892,34274045,\n \nWorld Bank.,Does not contain dataset mention
4893,34274045,2020.,Does not contain dataset mention
4894,34274045,Enhancing Government Effectiveness and Transpa...,Does not contain dataset mention


## Automating Model Push to HuggingFace Hub

In [2]:
from transformers import BertForSequenceClassification

base_path = '/content/drive/Shareddrives/World Bank/NLP Project/'
model_name = 'BERT_FineTuned_1'
model_save_path = base_path + "Trained Models/" + model_name

# Load the model back
model = BertForSequenceClassification.from_pretrained(model_save_path)

In [3]:
## Code will not work on Colab as cannot initialize a Git repo

from huggingface_hub import HfFolder, Repository
import os
import tempfile

# Prompt the user to enter their Hugging Face username and access token
hf_username = input("Enter your Hugging Face username: ")
hf_access_token = input("Enter your Hugging Face access token: ")

# Desired model name on the hub
model_name_on_hub = 'DataBERT'

# Ensure your access token is used for authentication
HfFolder.save_token(hf_access_token)

# Create a temporary directory to store the model and tokenizer files
temp_dir = tempfile.mkdtemp()

# Initialize an empty repository in the temporary directory
repo = Repository(temp_dir, clone_from=f"{hf_username}/{model_name_on_hub}", use_auth_token=hf_access_token)

# Save the model and tokenizer in the temporary directory
model.save_pretrained(temp_dir)

# Add, commit, and push the files to the Hugging Face Hub
repo.push_to_hub(commit_message="Deploy DataBERT and tokenizer to Hugging Face Hub")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)


OSError: Tried to clone a repository in a non-empty folder that isn't a git repository ('/content/hf_model'). If you really want to do this, do it manually:
 cd /content/hf_model && git init && git remote add origin && git pull origin main
 or clone repo to a new folder and move your existing files there afterwards.